<a href="https://colab.research.google.com/github/ShreyasHavaldar7/DL-Applications/blob/master/GCN/Basic_GCN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
from torch_geometric.datasets import Planetoid
dataset = Planetoid(root='/tmp/Cora', name='Cora')

In [0]:
pip install torch-geometric

In [0]:
!pip install torch-scatter
!pip install torch-sparse
!pip install torch-cluster
!pip install torch-spline-conv 

In [0]:
!nvcc --version

In [0]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

In [0]:
class Net(torch.nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.conv1 = GCNConv(dataset.num_node_features, 16)
    self.conv2 = GCNConv(16, dataset.num_classes)
  def forward(self, data):
    x, edge_index = data.x, data.edge_index

    x = self.conv1(x, edge_index)
    x = F.relu(x)
    x = F.dropout(x, training = self.training)
    x = self.conv2(x, edge_index)

    return F.log_softmax(x, dim=1)

In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

In [0]:
!pip install gputil
!pip install psutil
!pip install humanize

import GPUtil as GPU
import os
import humanize
import psutil


GPUs = GPU.getGPUs()
gpu= GPUs[0]

def printer():
  process = psutil.Process(os.getpid())
  print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available),
          " | Proc size: " + humanize.naturalsize(process.memory_info().rss))
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(
        gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
    
   
printer()  
#checking alloted gpu by colab

In [0]:
!nvidia-smi

In [0]:
import matplotlib.pyplot as plt
model = Net().to(device)
data = dataset[0].to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 0.01, weight_decay= 7e-4)

model.train()
for epoch in range(500):
  optimizer.zero_grad()
  out = model(data)
  loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
  print('Epoch: {} | Loss: {:.7f} ' .format(epoch, loss.item()) )
  loss.backward()
  optimizer.step()


In [0]:
model.eval()
_, pred = model(data).max(dim=1)
correct = float(pred[data.test_mask].eq(data.y[data.test_mask]).sum().item())
acc = correct/data.test_mask.sum().item()
print('Accuracy: {:.6f}'.format(acc))